# A Few Approaches to Quadratic Parameter Estimation

This notebook is an extension of the FastAI course's "How does a neural net really work?" notebook that introduces users to `autograd` and `Tensor`.  To best get a feel for the lowest level of PyTorch after working with Lightning for some time, this is a quick trip back to the fundamentals.

This notebook is split into the following sections:

1. Define a few functions to build arbitrary quadratic functions and samplers that will add some noise to simulate noisy data derived from these functions
2. Classic quadratic regression using the Moore-Penrose pseudoinverse to find the solution to the least squares problem.  This will serve as the gold standards for other methods
3. Three parameter model using `torch` and attempt to find the quadratic parameters through gradient descent
4. The same three parameter model using closed form solutions for the gradient of the loss function
5. Using `scipy` to leverage some well designed optimizers to solve the problem

In [ ]:
!pip install torch numpy matplotlib scipy 

In [ ]:
import torch
import numpy as np
from matplotlib import pyplot as plt
import scipy.optimize

In [ ]:
def quadratic_factory(a: float, b: float, c: float) -> callable:
    """Build a function that will return values defined by the quadratic function
    parametrized by a, b, and c

    Args:
        a (float): The squared coefficient
        b (float): The linear coefficient
        c (float): The constant value

    Returns:
        callable: A function that returns 'y' values for the quadratic function
            parametrized by a, b, and c
    """

    def quadratic(x: float | np.ndarray) -> float | np.ndarray:
        """The quadratic function"""
        return a * x**2 + b * x + c

    return quadratic


def noisy_samples(quadratic: callable, sigma: float) -> callable:
    """Wrap a defined quadratic function with a noise generator
    to generate training samples

    Args:
        quadratic (callable): The defined quadratic function, see above
        sigma (float): The variance of the gaussian noise added to every sample

    Returns:
        callable: A function that returns noisy samples of specified shape
    """

    def noisy_quad(x: float | np.ndarray):
        return quadratic(x) + np.random.randn(*x.shape) * sigma

    return noisy_quad

In [ ]:
# Define a function and plot it
a = 4
b = 3
c = -1
quad_func = quadratic_factory(a, b, c)
sampler = noisy_samples(quadratic=quad_func, sigma=5)

x = np.linspace(-5, 5, 50)
y = quad_func(x)

# Generate some samples between [-5, 5]
x_sampled = (np.random.rand(50) - 0.5) * 10
y_sampled = sampler(x_sampled)

plt.plot(x, y, label="F(x)")
plt.plot(x_sampled, y_sampled, "r.", label="Sampled F(x)")
plt.title(f"f(x) = ${a}x^2 + {b}x + {c}$")
plt.ylabel("$f(x)$")
plt.xlabel("$x$")
plt.legend()
plt.show()

# Quadratic Regression

The problem we are trying to solve -- finding the optimal coefficients for an unknown quadratic -- can be solved closed-form using simple regression.  However, inverting matrices takes time and more memory than a 3 parameter gradient descent.  Let's draw a baseline with regression and see how it stacks up to gradient descent with a parameter sweep of number of samples and amount of noise.

In [ ]:
def quadratic_regression(x: np.ndarray, y: np.ndarray) -> dict:
    """Run regression on noisy data to minimize the mean squared error
    over three parameters

    Args:
        x (np.ndarray): The input x coordinates
        y (np.ndarray): The input y coordinates

    Returns:
        dict: information summarizing the regression
    """
    matrix_x = np.vstack((x**2, x, np.ones_like(x))).T
    matrix_x_inv = np.dot(np.linalg.inv(np.dot(matrix_x.T, matrix_x)), matrix_x.T)
    best_fit = np.dot(matrix_x_inv, y)
    y_predict = best_fit[0] * x**2 + best_fit[1] * x + best_fit[2]
    cost = np.mean((y_predict - y) ** 2)
    return dict(cost=cost, fit_params=best_fit)

# Gradient Descent

Instead of inverting matrices, we could just iterate many times to try to find a set of optimal parameters with the same mean squared error cost function.  While memory is not a realistic constraint, this method does have the advantage of consuming significantly less memory than matrix inversion at the cost of iteration time.

In [ ]:
def gradient_descent(
    x: np.ndarray, y: np.ndarray, learning_rate: float = 1e-3, max_iterations: int = 100
) -> dict:
    """Given samples of a quadratic function, find optimal parameters with gradient descent using 
    torch's autograd

    Args:
        x (np.ndarray): The inputs to the function we are attempting to approximate
        y (np.ndarray): The targets we are going to minimize error against
        learning_rate (float, optional): The learning rate to multiply gradients by. Defaults to 1e-3.
        max_iterations (int, optional): The maximum number of iterations before terminating the algorithm. Defaults to 100.

    Returns:
        dict: Optimization outputs such as a gradient, parameter, and cost histories as well as the best parameters found
    """
    # X and Y must be tensors to work with torch
    x = torch.Tensor(x)
    y = torch.Tensor(y)
    # Randomly initialize our a, b, and c guesses to start
    parameters = torch.ones(3)
    parameters.requires_grad = True

    # Track the best set of parameters found so far
    best_parameters = None
    best_cost = np.inf
    
    # Track iteration count and values of key quantities for each iteration
    iteration = 0
    costs = []
    grads = []
    params = []

    while iteration < max_iterations:
        y_predict = parameters[0] * x**2 + parameters[1] * x + parameters[2]
        cost = torch.square(y_predict - y).mean()
        cost.backward()
        print(f"Iter {iteration} cost: {cost}")
        with torch.no_grad():
            gradient = parameters.grad
            parameters -= gradient * learning_rate
        grads.append(np.array(parameters.grad.detach()))
        costs.append(float(cost))
        params.append(np.array(parameters.detach()))

        if costs[-1] < best_cost:
            best_cost = costs[-1]
            best_parameters = params[-1]

        iteration += 1

    return dict(
        cost=cost,
        costs=costs,
        grads=grads,
        params=params,
        fit_params=best_parameters,
        iterations=iteration,
    )

In [ ]:
def plot_result(x, y, targets: np.ndarray, opt_params: np.ndarray):
    target_func = quadratic_factory(*targets)
    fit_func = quadratic_factory(*opt_params)
    print(f"f`(x) = {opt_params[0]}x^2 + {opt_params[1]}x + {opt_params[2]}")

    # Compute MSE
    mse = np.mean((fit_func(x) - y) ** 2)
    print(f"Mean Squared Error: {mse}")

    plt.plot(x, target_func(x), label="F(x)")
    plt.plot(x, y, "r.", label="Sampled F(x)")
    plt.plot(x, fit_func(x), "k", label="F`(x)")
    plt.title(f"f(x) = ${targets[0]}x^2 + {targets[1]}x + {targets[2]}$")
    plt.ylabel("$y$")
    plt.xlabel("$x$")
    plt.legend()
    plt.show()

In [ ]:
n_samples = 250
sigma = 1
np.random.seed(30)
targets = np.round(np.random.rand(3) - .5, 3)
x = (np.random.rand(n_samples) - 0.5) * 10
x.sort()
target_func = quadratic_factory(*targets)
sampler = noisy_samples(target_func, sigma=sigma)
y = sampler(x)

qr_result = quadratic_regression(x, y)
plot_result(x, y, targets=targets, opt_params=qr_result['fit_params'])

gd_result = gradient_descent(x, y, learning_rate=1e-4)
plot_result(x, y, targets=targets, opt_params=gd_result['fit_params'])

# Gradient Descent Problems

The two differing results above show gradient descent is not a good way to solve this problem (unless gradient descent got very lucky)!  As the number of points gets very large, the quadratic regression solution's loss term will tend toward the variance of the noise - a perfect result considering the noise is uncorrelated.  The gradient descent solution will "learn" a better set of parameters than the ones it started with, but will be mostly wrong.  Plotting the gradients and cost may lend some insight

In [ ]:
plt.plot(gd_result['costs'])
plt.ylabel('Cost')
plt.xlabel('Epoch')
plt.title('Cost over Training')
plt.figure()
a_grad, b_grad, c_grad = list(zip(*gd_result['grads']))
plt.plot(a_grad, label='Grad A')
plt.plot(b_grad, label='Grad B')
plt.plot(c_grad, label='Grad C')
plt.legend()
plt.ylabel('Gradient Magnitude')
plt.xlabel('Epoch')
plt.title('Gradient over Training')
plt.show()

It looks like our cost function is oscillating during training, which means our step size is too big and is overshooting the global minimum.  We know there is a global minimum as we have a closed form, unique solution to our regression problem, so why doesn't gradient descent find it?  It looks like our gradients have huge differences in magnitude, which may have something to do with it.  Perhaps small changes in `a` dominate the cost function and the correct value of `a` is located at the bottom of a very narrow valley which would require a very slow learning rate to traverse.

Fortunately, we can compute closed form solutions for the first order partial derivatives of our cost function to see if approximations are the problem:

$$l(a, b, c) = \sum_{i=1}^{N} ((ax_{i}^{2} + bx + c) - y_i^2)$$


$$\frac{dl}{da} = \frac{2}{n} \sum_{i = 1}^{N} x_i^2((ax_i^2 + bx_i + c) - y_i)$$
$$\frac{dl}{db} = \frac{2}{n} \sum_{i = 1}^{N} x_i((ax_i^2 + bx_i + c) - y_i)$$
$$\frac{dl}{dc} = \frac{2}{n} \sum_{i = 1}^{N} ax_i^2 + bx_i + c - y_i$$



In [ ]:
def known_gradient_descent(
    x: np.ndarray, y: np.ndarray, learning_rate: float = 1e-3, max_iterations: int = 100
) -> dict:
    """Given samples of a quadratic function, find optimal parameters with gradient descent using precomputed gradient functions"""
    # Randomly initialize our a, b, and c guesses to start
    a, b, c = np.random.randn(3)

    # Track the best set of parameters found so far
    best_parameters = None
    best_cost = np.inf
    
    # Track iteration count and values of key quantities for each iteration
    iteration = 0
    costs = []
    grads = {'a':[], 'b': [], 'c': []}
    params = []
    while iteration < max_iterations:
        y_predict = a * x**2 + b * x + c
        cost = np.square(y_predict - y).mean()
        print(f"Iter {iteration} cost: {cost}")

        grad_a = 2 * np.mean(x**2 * ((a * x ** 2 + b * x + c) - y))
        grad_b = 2 * np.mean(x* ((a * x ** 2 + b * x + c) - y))
        grad_c = 2 * np.mean(a * x ** 2 + b * x + c - y)

        grads['a'].append(grad_a)
        grads['b'].append(grad_b)
        grads['c'].append(grad_c)

        # Update the parameters
        a -= grad_a * learning_rate
        b -= grad_b * learning_rate
        c -= grad_c * learning_rate

        costs.append(float(cost))
        params.append(np.array([a, b, c]))

        if costs[-1] < best_cost:
            best_cost = costs[-1]
            best_parameters = params[-1]

        iteration += 1

    return dict(
        cost=cost,
        costs=costs,
        grads=grads,
        params=params,
        fit_params=best_parameters,
        iterations=iteration,
    )

In [ ]:
kgd_result = known_gradient_descent(x, y, learning_rate=1e-3, max_iterations=150)
plot_result(x, y, targets=targets, opt_params=kgd_result['fit_params'])

In [ ]:
plt.plot(kgd_result['costs'])
plt.ylabel('Cost')
plt.xlabel('Epoch')
plt.title('Cost over Training')
plt.figure()
a_grad, b_grad, c_grad = [list(x) for x in kgd_result['grads'].values()]
plt.plot(a_grad, label='Grad A')
plt.plot(b_grad, label='Grad B')
plt.plot(c_grad, label='Grad C')
plt.legend()
plt.ylabel('Gradient Magnitude')
plt.xlabel('Epoch')
plt.title('Gradient over Training')
plt.show()

It looks like a closed form solution to the gradient helped with smoothing out our costs and gradients to a point where a steady state was reached after about 100 epochs.  After playing with the learning rate, it looks like .001 is a decent value but it still takes some time for `c` to converge.  There are some lingering instabilities with higher learning rates - try out a learning rate of .1 and gradients should explode and crash the optimization.  Unforunately, even after all that, we still don't have a very good solution compared with our closed form solution.

# Scipy Optimization

While the solution above provides a somewhat satisfying alternative to the problem with original gradient descent, we might as well see what a more carefully designed optimizer can achieve in our problem.  For this, we can look at the `scipy.optimize` package and utilize a host of solvers to try to tackle the problem.

In [ ]:
def objective(parameters: np.ndarray) -> float:
    """Define the cost function for optimization"""
    return np.square((parameters[0] * x ** 2 + parameters[1] * x + parameters[2]) - y).mean()

def jacobian(parameters: np.ndarray) -> np.ndarray:
    """Define the jacobian (vector of partial derivatives) for optimization, same as known_gradient_descent"""
    y_prime = parameters[0] * x ** 2 + parameters[1] * x + parameters[2]
    da = 2 * np.mean(x ** 2 * (y_prime - y))
    db = 2 * np.mean(x * (y_prime - y))
    dc = 2 * np.mean(y_prime - y)
    return np.array([da, db, dc])

# Random parameter initialization
params = np.random.randn(3)
result = scipy.optimize.minimize(fun=objective, x0=params, method='BFGS', jac=jacobian)
print(result)
plot_result(x, y, targets=targets, opt_params=result.x)

This solution is virtually identical to our closed form matrix solution and it converges in less than 10 iterations.

# Conclusions

While this was presented as the "easy intro" to gradient descent in the fast.ai course, it reveals serious shortcomings of gradient descent when applied to some problems.  There are plenty of other means to solve the quadratic regression (or to generalize, polynomial regression) problem and the method proposed in the course is possibly the worst one an engineer could choose.  While simple to implement and lightweight on memory, this method relies on a very sensitive learning rate parameter to get things right, and the solutions will be far from optimal.